In [ ]:
#This is for INSY660 GROUP13 PROJECT

#We decided to shift gear and design a chatbot for Chinese restaurant owners.
#This chatbot can help recommend Chinese dishes to customers who are not familiar with Chinese dish.
#With the customer input, the restaurant owners will have a better customer database, which will be helpful for future decision makings.
#Since we have not found any relavent databases online that fits our purposes, we are creating a mock data as our recommendation fundation.

In [ ]:
#Import data
import pandas as pd
import csv
import numpy as np

!pip install XlsxWriter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from openpyxl import load_workbook

from datetime import datetime

pd.set_option('display.max_rows', None)
from google.colab import drive
drive.mount('/content/gdrive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 2.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
#Import the historical data which will be used in the recommendation model.
home_dir = "/content/gdrive/MyDrive/Colab Notebooks/INSY660_GROUP_PROJECT/"
df1 = pd.read_csv(home_dir+'chinese_food_dataset.csv')
df1

,bill_ID,name,age,gender,nationality,number of people,love spicy?,favorate Chinese dishes,billing amount
0,1,Customer_1,28,Female,Brazil,1,Yes,Pineapple Fried Rice,6.99
1,2,Customer_2,44,Female,India,2,No,Sweet and Sour Pork,23.98
2,3,Customer_3,52,Female,Mexico,6,No,Shrimp Lo Mein,59.94
3,4,Customer_4,36,Male,United States,2,No,Crispy Duck,31.98
4,5,Customer_5,30,Female,Mexico,10,No,Szechuan Beef,129.90
5,6,Customer_6,51,Female,Spain,7,No,Kung Pao Chicken,76.93
6,7,Customer_7,48,Female,Russia,1,No,Beef Chow Fun,8.99
7,8,Customer_8,21,Female,Brazil,10,No,Dan Dan Noodles,79.90
8,9,Customer_9,47,Female,China,8,Yes,Kung Pao Chicken,87.92
9,10,Customer_10,59,Male,Switzerland,8,No,Crispy Duck,127.92


In [ ]:
#1.data analysis
#British,50-60,male,spciy,Fried Rice
#100 British,(60%),Fried Rice
#2.chatbot:
#   a.where are you from?
#   b.your age?
#   c.do you love spicy food?
#   d.how many people do you have?
#   f.Do you have any dietary restriction?(nuts,lactose intolerance)
#   g.Are you vegeterian
#   h.what is your budget?
#3.data structure
# class: dishes
#     a.price
#     b.name
#     c.spicy
#     a.>5,suggest
#     b.nuts
#     c.lactose intolerance
#     d.vegeterian
#       1.soup
#       2.main
#       3.dishes


In [ ]:
#1.data analysis
df = df1.copy()
df = df.drop(['bill_ID', 'name'], axis=1)
df['bucket_age'] = df['age'].map(lambda x: 1 if x<=30 else ( 2 if x<=50 else 3 ))
df = df.drop(['age'], axis=1)
# The count of favorate Chinese dishes of every nationality
d = df[['nationality', 'favorate Chinese dishes', 'number of people']].groupby(['nationality', 'favorate Chinese dishes']).apply(lambda x: x['number of people'].sum())
d = d.to_frame()
d


0
nationality    favorate Chinese dishes      
Australia      Beef Chow Fun              74
               Broccoli Beef              76
               Chicken with Cashew Nuts   70
               Crispy Duck                72
               Dan Dan Noodles            67
               Egg Drop Soup              62
               General Tso's Chicken      66
               Hot and Sour Soup          46
               Kung Pao Chicken           80
               Ma Po Tofu                 56
               Mapo Tofu                  35
               Mongolian Beef             84
               Orange Chicken             62
               Pineapple Fried Rice       73
               Shrimp Lo Mein            112
               Sweet and Sour Pork       101
               Szechuan Beef              51
               Vegetable Fried Rice       83
               Wonton Soup                49
               Yangzhou Fried Rice        59
Brazil         Beef Chow Fun              65
               Broccoli Beef              93
               Chicken with Cashew Nuts   88
               Crispy Duck                64
               Dan Dan Noodles            63
               Egg Drop Soup              74
               General Tso's Chicken     120
               Hot and Sour Soup          90
               Kung Pao Chicken           59
               Ma Po Tofu                110
               Mapo Tofu                  36
               Mongolian Beef             70
               Orange Chicken             58
               Pineapple Fried Rice       53
               Shrimp Lo Mein             68
               Sweet and Sour Pork        99
               Szechuan Beef             102
               Vegetable Fried Rice       66
               Wonton Soup                40
               Yangzhou Fried Rice        67
Canada         Beef Chow Fun              82
               Broccoli Beef              36
               Chicken with Cashew Nuts   58
               Crispy Duck                59
               Dan Dan Noodles            27
               Egg Drop Soup              59
               General Tso's Chicken      85
               Hot and Sour Soup          61
               Kung Pao Chicken           54
               Ma Po Tofu                 42
               Mapo Tofu                  69
               Mongolian Beef             95
               Orange Chicken             37
               Pineapple Fried Rice       63
               Shrimp Lo Mein             59
               Sweet and Sour Pork        34
               Szechuan Beef              79
               Vegetable Fried Rice       76
               Wonton Soup                77
               Yangzhou Fried Rice        49
China          Beef Chow Fun              76
               Broccoli Beef             107
               Chicken with Cashew Nuts   70
               Crispy Duck                64
               Dan Dan Noodles            33
               Egg Drop Soup              94
               General Tso's Chicken      64
               Hot and Sour Soup          40
               Kung Pao Chicken           73
               Ma Po Tofu                 56
               Mapo Tofu                  35
               Mongolian Beef             72
               Orange Chicken             80
               Pineapple Fried Rice       80
               Shrimp Lo Mein             54
               Sweet and Sour Pork       104
               Szechuan Beef              69
               Vegetable Fried Rice       53
               Wonton Soup                78
               Yangzhou Fried Rice        67
France         Beef Chow Fun              56
               Broccoli Beef              41
               Chicken with Cashew Nuts   68
               Crispy Duck               105
               Dan Dan Noodles            44
               Egg Drop Soup              66
               General Tso's Chicken      32
               Hot and Sour Soup      

In [ ]:
# The count of love spicy of every nationality
d = df[['nationality', 'love spicy?', 'number of people']].groupby(['nationality', 'love spicy?']).apply(lambda x: x['number of people'].sum())
d = d.to_frame()
d

0
nationality    love spicy?     
Australia      No           767
               Yes          611
Brazil         No           822
               Yes          663
Canada         No           573
               Yes          628
China          No           713
               Yes          656
France         No           617
               Yes          598
Germany        No           763
               Yes          683
India          No           751
               Yes          609
Italy          No           773
               Yes          720
Japan          No           738
               Yes          665
Mexico         No           651
               Yes          712
Netherlands    No           618
               Yes          653
Russia         No           690
               Yes          754
Singapore      No           650
               Yes          778
South Korea    No           844
               Yes          621
Spain          No           841
               Yes          685
Sweden         No           741
               Yes          663
Switzerland    No           682
               Yes          762
Turkey         No           654
               Yes          647
United Kingdom No           659
               Yes          719
United States  No           836
               Yes          707

In [ ]:
#The count of love spicy of every gender and every nationality
d = df[['nationality', 'gender', 'love spicy?', 'number of people']].groupby(['nationality', 'gender', 'love spicy?']).apply(lambda x: x['number of people'].sum())
d = d.to_frame()
d

0
nationality    gender love spicy?     
Australia      Female No           361
                      Yes          300
               Male   No           406
                      Yes          311
Brazil         Female No           394
                      Yes          317
               Male   No           428
                      Yes          346
Canada         Female No           269
                      Yes          361
               Male   No           304
                      Yes          267
China          Female No           361
                      Yes          357
               Male   No           352
                      Yes          299
France         Female No           317
                      Yes          347
               Male   No           300
                      Yes          251
Germany        Female No           332
                      Yes          383
               Male   No           431
                      Yes          300
India          Female No           366
                      Yes          264
               Male   No           385
                      Yes          345
Italy          Female No           318
                      Yes          334
               Male   No           455
                      Yes          386
Japan          Female No           335
                      Yes          351
               Male   No           403
                      Yes          314
Mexico         Female No           299
                      Yes          411
               Male   No           352
                      Yes          301
Netherlands    Female No           275
                      Yes          338
               Male   No           343
                      Yes          315
Russia         Female No           333
                      Yes          313
               Male   No           357
                      Yes          441
Singapore      Female No           376
                      Yes          375
               Male   No           274
                      Yes          403
South Korea    Female No           474
                      Yes          357
               Male   No           370
                      Yes          264
Spain          Female No           417
                      Yes          383
               Male   No           424
                      Yes          302
Sweden         Female No           428
                      Yes          355
               Male   No           313
                      Yes          308
Switzerland    Female No           329
                      Yes          408
               Male   No           353
                      Yes          354
Turkey         Female No           311
                      Yes          330
               Male   No           343
                      Yes          317
United Kingdom Female No           355
                      Yes          351
               Male   No           304
                      Yes          368
United States  Female No           417
                      Yes          371
               Male   No           419
                      Yes          336

In [ ]:
# The count of love spicy of every nationality and age group
d = df[['nationality', 'bucket_age', 'love spicy?', 'number of people']].groupby(['nationality', 'bucket_age', 'love spicy?']).apply(lambda x: x['number of people'].sum())
d = d.to_frame()
d

0
nationality    bucket_age love spicy?     
Australia      1          No           205
                          Yes          192
               2          No           363
                          Yes          240
               3          No           199
                          Yes          179
Brazil         1          No           230
                          Yes          176
               2          No           330
                          Yes          248
               3          No           262
                          Yes          239
Canada         1          No           161
                          Yes          118
               2          No           228
                          Yes          277
               3          No           184
                          Yes          233
China          1          No           199
                          Yes          177
               2          No           291
                          Yes          287
               3          No           223
                          Yes          192
France         1          No           161
                          Yes          161
               2          No           263
                          Yes          267
               3          No           193
                          Yes          170
Germany        1          No           187
                          Yes          182
               2          No           353
                          Yes          281
               3          No           223
                          Yes          220
India          1          No           217
                          Yes          150
               2          No           360
                          Yes          196
               3          No           174
                          Yes          263
Italy          1          No           242
                          Yes          214
               2          No           337
                          Yes          308
               3          No           194
                          Yes          198
Japan          1          No           222
                          Yes          134
               2          No           269
                          Yes          255
               3          No           247
                          Yes          276
Mexico         1          No           210
                          Yes          214
               2          No           249
                          Yes          259
               3          No           192
                          Yes          239
Netherlands    1          No           161
                          Yes          173
               2          No           229
                          Yes          294
               3          No           228
                          Yes          186
Russia         1          No           167
                          Yes          170
               2          No           287
                          Yes          301
               3          No           236
                          Yes          283
Singapore      1          No           193
                          Yes          159
               2          No           227
                          Yes          347
               3          No           230
                          Yes          272
South Korea    1          No           244
                          Yes          178
               2          No           317
                          Yes          244
               3          No           283
                          Yes          199
Spain          1          No           229
                          Yes          206
               2          No           323
                          Yes          281
               3          No           289
                          Yes          198
Sweden         1          No           229
                          Yes          197

In [ ]:
# The count of favorate Chinese dishes of every gender and age group
d = df[['gender', 'bucket_age', 'favorate Chinese dishes', 'number of people']].groupby(['gender', 'bucket_age', 'favorate Chinese dishes']).apply(lambda x: x['number of people'].sum())
d = d.to_frame()
d

0
gender bucket_age favorate Chinese dishes      
Female 1          Beef Chow Fun             115
                  Broccoli Beef             230
                  Chicken with Cashew Nuts  111
                  Crispy Duck               265
                  Dan Dan Noodles           194
                  Egg Drop Soup             206
                  General Tso's Chicken     250
                  Hot and Sour Soup         174
                  Kung Pao Chicken          170
                  Ma Po Tofu                162
                  Mapo Tofu                 129
                  Mongolian Beef            218
                  Orange Chicken            194
                  Pineapple Fried Rice      185
                  Shrimp Lo Mein            161
                  Sweet and Sour Pork       251
                  Szechuan Beef             207
                  Vegetable Fried Rice      146
                  Wonton Soup               149
                  Yangzhou Fried Rice       149
       2          Beef Chow Fun             362
                  Broccoli Beef             378
                  Chicken with Cashew Nuts  313
                  Crispy Duck               257
                  Dan Dan Noodles           281
                  Egg Drop Soup             246
                  General Tso's Chicken     271
                  Hot and Sour Soup         211
                  Kung Pao Chicken          173
                  Ma Po Tofu                312
                  Mapo Tofu                 231
                  Mongolian Beef            272
                  Orange Chicken            257
                  Pineapple Fried Rice      293
                  Shrimp Lo Mein            287
                  Sweet and Sour Pork       347
                  Szechuan Beef             225
                  Vegetable Fried Rice      270
                  Wonton Soup               271
                  Yangzhou Fried Rice       249
       3          Beef Chow Fun             157
                  Broccoli Beef             283
                  Chicken with Cashew Nuts  243
                  Crispy Duck               271
                  Dan Dan Noodles           242
                  Egg Drop Soup             240
                  General Tso's Chicken     189
                  Hot and Sour Soup         295
                  Kung Pao Chicken          259
                  Ma Po Tofu                281
                  Mapo Tofu                 248
                  Mongolian Beef            287
                  Orange Chicken            221
                  Pineapple Fried Rice      337
                  Shrimp Lo Mein            171
                  Sweet and Sour Pork       249
                  Szechuan Beef             211
                  Vegetable Fried Rice      185
                  Wonton Soup               264
                  Yangzhou Fried Rice       268
Male   1          Beef Chow Fun             185
                  Broccoli Beef             138
                  Chicken with Cashew Nuts  175
                  Crispy Duck               128
                  Dan Dan Noodles           156
                  Egg Drop Soup             243
                  General Tso's Chicken     184
                  Hot and Sour Soup         218
                  Kung Pao Chicken          196
                  Ma Po Tofu                221
                  Mapo Tofu                 215
                  Mongolian Beef            153
                  Orange Chicken            251
                  Pineapple Fried Rice      194
                  Shrimp Lo Mein            164
                  Sweet and Sour Pork       149
                  Szechuan Beef             209
                  Vegetable Fried Rice      207
                  Wonton Soup               178
                  Yangzhou Fried Rice       208
       2          Beef Chow Fun             266
                  Broccoli Beef             294
              

In [ ]:
d = df[['love spicy?', 'favorate Chinese dishes', 'number of people']].groupby(['favorate Chinese dishes', 'love spicy?']).apply(lambda x: x['number of people'].sum()).rename("sum")
d = d.to_frame().reset_index()
d0 = d.groupby('favorate Chinese dishes').apply(lambda x: (x[x['sum'] == x['sum'].max()]).iloc[0, :])
d0 = d0[['love spicy?']].reset_index()
d0

,favorate Chinese dishes,love spicy?
0,Beef Chow Fun,No
1,Broccoli Beef,Yes
2,Chicken with Cashew Nuts,Yes
3,Crispy Duck,No
4,Dan Dan Noodles,No
5,Egg Drop Soup,Yes
6,General Tso's Chicken,Yes
7,Hot and Sour Soup,No
8,Kung Pao Chicken,Yes
9,Ma Po Tofu,Yes


In [ ]:
d = df[['gender', 'favorate Chinese dishes', 'number of people']].groupby(['favorate Chinese dishes', 'gender']).apply(lambda x: x['number of people'].sum()).rename("sum")
d = d.to_frame().reset_index()
d1 = d.groupby('favorate Chinese dishes').apply(lambda x: (x[x['sum'] == x['sum'].max()]).iloc[0, :])
d1 = d1[['gender']].reset_index()
d1

,favorate Chinese dishes,gender
0,Beef Chow Fun,Male
1,Broccoli Beef,Female
2,Chicken with Cashew Nuts,Female
3,Crispy Duck,Female
4,Dan Dan Noodles,Female
5,Egg Drop Soup,Male
6,General Tso's Chicken,Female
7,Hot and Sour Soup,Female
8,Kung Pao Chicken,Male
9,Ma Po Tofu,Male


In [ ]:
d = df[['nationality', 'favorate Chinese dishes', 'number of people']].groupby(['favorate Chinese dishes', 'nationality']).apply(lambda x: x['number of people'].sum()).rename("sum")
d = d.to_frame().reset_index()
d2 = d.groupby('favorate Chinese dishes').apply(lambda x: (x[x['sum'] == x['sum'].max()]).iloc[0, :])
d2 = d2[['nationality']].reset_index()
d2

,favorate Chinese dishes,nationality
0,Beef Chow Fun,South Korea
1,Broccoli Beef,South Korea
2,Chicken with Cashew Nuts,Switzerland
3,Crispy Duck,Switzerland
4,Dan Dan Noodles,India
5,Egg Drop Soup,China
6,General Tso's Chicken,Brazil
7,Hot and Sour Soup,Singapore
8,Kung Pao Chicken,Spain
9,Ma Po Tofu,Brazil


In [ ]:
d = df[['bucket_age', 'favorate Chinese dishes', 'number of people']].groupby(['favorate Chinese dishes', 'bucket_age']).apply(lambda x: x['number of people'].sum()).rename("sum")
d = d.to_frame().reset_index()
d3 = d.groupby('favorate Chinese dishes').apply(lambda x: (x[x['sum'] == x['sum'].max()]).iloc[0, :])
d3 = d3[['bucket_age']].reset_index()

In [ ]:
d1 = d1.set_index('favorate Chinese dishes')
d2 = d2.set_index('favorate Chinese dishes')
d3 = d3.set_index('favorate Chinese dishes')
d0 = d0.set_index('favorate Chinese dishes')
d_food_attri = d1.join(d2).join(d3)
#d = d.join(d0)
d_food_attri

,gender,nationality,bucket_age
favorate Chinese dishes,,,
Beef Chow Fun,Male,South Korea,2
Broccoli Beef,Female,South Korea,2
Chicken with Cashew Nuts,Female,Switzerland,2
Crispy Duck,Female,Switzerland,2
Dan Dan Noodles,Female,India,2
Egg Drop Soup,Male,China,2
General Tso's Chicken,Female,Brazil,2
Hot and Sour Soup,Female,Singapore,3
Kung Pao Chicken,Male,Spain,3


In [ ]:
d_food_attri["attribute"] = d_food_attri['gender'].astype(str) +"|"+ d_food_attri["nationality"].astype(str)+"|"+ d_food_attri["bucket_age"].astype(str)
d_food_attri

,gender,nationality,bucket_age,attribute
favorate Chinese dishes,,,,
Beef Chow Fun,Male,South Korea,2,Male|South Korea|2
Broccoli Beef,Female,South Korea,2,Female|South Korea|2
Chicken with Cashew Nuts,Female,Switzerland,2,Female|Switzerland|2
Crispy Duck,Female,Switzerland,2,Female|Switzerland|2
Dan Dan Noodles,Female,India,2,Female|India|2
Egg Drop Soup,Male,China,2,Male|China|2
General Tso's Chicken,Female,Brazil,2,Female|Brazil|2
Hot and Sour Soup,Female,Singapore,3,Female|Singapore|3
Kung Pao Chicken,Male,Spain,3,Male|Spain|3


In [ ]:
#3.data structure
# Design Pattern: Factory Method Pattern:
class Dish:
    def __init__(self, name, price, is_spicy, category, is_nuts, is_lact, is_vege,introduction):
        self.name = name
        self.price = price
        self.is_spicy = is_spicy
        self.category = category
        self.is_nuts = is_nuts
        self.is_lact = is_lact
        self.is_vege = is_vege
        self.introduction = introduction

class DishFactory:
    def create_dish(self, name, price, is_spicy, category, is_nuts, is_lact, is_vege,introduction):
        return Dish(name, price, is_spicy, category, is_nuts, is_lact, is_vege,introduction)

class Menu:
    def __init__(self):
        self.dishes = []

    def add_dish(self, dish):
        self.dishes.append(dish)

    def display_menu(self):
        for dish in self.dishes:
            print(f"Name: {dish.name}")
            print(f"Price: ${dish.price}")
            print(f"Spicy: {'Yes' if dish.is_spicy else 'No'}")
            print(f"Category: {dish.category}")
            print(f"nuts contained: {'Yes' if dish.is_nuts else 'No'}")
            print(f"lactose contained: {'Yes' if dish.is_lact else 'No'}")
            print(f"Is this dish suitable for vegetarians: {'Yes' if dish.is_vege else 'No'}")
            print(f"Introduction: {dish.introduction}")
            print("-" * 20)


In [ ]:
soups = ["Hot and Sour Soup", "Wonton Soup", "Egg Drop Soup"]
noodles = ["Beef Chow Fun", "Dan Dan Noodles", "Shrimp Lo Mein"]
fried_rice = ["Yangzhou Fried Rice", "Pineapple Fried Rice", "Vegetable Fried Rice"]
famous_dishes = [
    "Kung Pao Chicken", "Mapo Tofu", "Sweet and Sour Pork",
    "General Tso's Chicken", "Mongolian Beef", "Ma Po Tofu",
    "Crispy Duck", "Szechuan Beef", "Broccoli Beef",
    "Chicken with Cashew Nuts", "Orange Chicken"
]
dish_prices = {
    "Hot and Sour Soup": 5.99,
    "Wonton Soup": 4.99,
    "Egg Drop Soup": 3.99,
    "Beef Chow Fun": 8.99,
    "Dan Dan Noodles": 7.99,
    "Shrimp Lo Mein": 9.99,
    "Yangzhou Fried Rice": 7.99,
    "Pineapple Fried Rice": 6.99,
    "Vegetable Fried Rice": 5.99,
    "Kung Pao Chicken": 10.99,
    "Mapo Tofu": 9.99,
    "Sweet and Sour Pork": 11.99,
    "General Tso's Chicken": 12.99,
    "Mongolian Beef": 13.99,
    "Ma Po Tofu": 9.99,
    "Crispy Duck": 15.99,
    "Szechuan Beef": 12.99,
    "Broccoli Beef": 10.99,
    "Chicken with Cashew Nuts": 11.99,
    "Orange Chicken": 12.99
}

total_prices = sum(dish_prices.values())
total_dishes = len(dish_prices)
average_price = total_prices / total_dishes

# Create dish objects using the Factory Method
dish_factory = DishFactory()

# Create soup dishes
soup_dishes = [dish_factory.create_dish(name, dish_prices[name], True, "Soup", False, False, False,'') for name in soups]

# Create noodle dishes
noodle_dishes = [dish_factory.create_dish(name, dish_prices[name], False, "Noodle", False, False, False,'') for name in noodles]

# Create fried rice dishes
fried_rice_dishes = [dish_factory.create_dish(name, dish_prices[name], False, "Fried Rice", False, False, False,'') for name in fried_rice]

# Create famous dish dishes
famous_dish_dishes = [dish_factory.create_dish(name, dish_prices[name], True, "Famous Dish", False, False, False,'') for name in famous_dishes]

# Combine all dishes
all_dishes = soup_dishes + noodle_dishes + fried_rice_dishes + famous_dish_dishes


In [ ]:
all_dishes[0].is_vege = True
all_dishes[7].is_vege = True
all_dishes[8].is_vege = True
all_dishes[9].is_nuts = True
all_dishes[18].is_nuts = True

all_dishes[0].introduction = """
Hot and Sour Soup is a beloved Chinese soup renowned for its harmonious balance
of contrasting flavors. This comforting dish features a hearty broth that is
both spicy and tangy, achieved through the interplay of ingredients like vinegar
and chili. Loaded with various textures, it typically includes silky tofu,
crunchy wood ear mushrooms, tender bamboo shoots, and sometimes protein like
shredded chicken or pork. The soup's robust flavors and contrasting textures
make it a popular choice for those seeking a warming and invigorating culinary
experience."""
all_dishes[1].introduction = """
Wonton Soup is a classic Chinese soup that exudes comfort and tradition. At its
heart are delicate parcels of minced meat, often pork, and/or shrimp, encased in
thin wonton wrappers. These dumplings are cooked to perfection and served in a
clear, flavorful broth. The soup showcases the art of simplicity, allowing the
pure flavors of the tender wontons to shine alongside subtle hints of aromatic
herbs and spices. With every spoonful, you'll savor the harmonious blend of
textures and tastes, making Wonton Soup a timeless favorite for those seeking a
soul-nourishing dish."""
all_dishes[2].introduction = """
Egg Drop Soup is a delicate and soothing Chinese dish that embodies simplicity
at its finest. This soup features a clear and savory broth enriched with the
gentlest touch of beaten eggs, creating delicate ribbons that elegantly dance
through the liquid. Often garnished with chopped scallions, the soup's beauty
lies in its minimalist composition and the interplay between the smoothness of
the eggs and the comforting warmth of the broth. Egg Drop Soup is a go-to choice
for those seeking a light and nourishing option, perfect for both savoring the
moment and finding solace on cooler days."""
all_dishes[3].introduction = """
Beef Chow Fun is a classic Cantonese dish that brings together wide rice noodles,
tender slices of beef, and crisp bean sprouts in a savory stir-fry. This dish
boasts a delightful contrast of textures, with the smooth noodles and succulent
beef complemented by the freshness of bean sprouts. Enhanced with a flavorful
soy-based sauce, Beef Chow Fun captures the essence of Cantonese cuisine in
every delicious bite."""
all_dishes[4].introduction = """
Dan Dan Noodles are a signature Sichuan dish that delivers a bold and fiery
flavor experience. Thin noodles are bathed in a spicy, savory sauce made from
chili oil, Sichuan peppercorns, and minced pork, creating a symphony of heat and
numbing sensations. Topped with crushed peanuts and green onions, this dish
offers a tantalizing medley of textures and tastes, making it a favorite among
those who crave the intense flavors of Sichuan cuisine."""
all_dishes[5].introduction = """
Shrimp Lo Mein features succulent shrimp, tender noodles, and vibrant vegetables,
all wok-tossed in a savory sauce. It's a delightful stir-fry that delivers a
satisfying blend of flavors and textures."""
all_dishes[6].introduction = """
Yangzhou Fried Rice is a flavorful Chinese dish that combines fluffy rice with
eggs, succulent shrimp, and an assortment of vegetables. This fried rice variation
offers a delightful medley of ingredients, all expertly wok-tossed to create a
harmonious symphony of tastes and textures."""
all_dishes[7].introduction = ""
all_dishes[8].introduction = ""
all_dishes[9].introduction = """
Kung Pao Chicken is a classic Sichuan dish known for its bold flavors and fiery kick. Tender pieces of chicken are wok-fried with peanuts, bell peppers, and dried chili peppers, creating a harmonious balance of sweet, spicy, and savory notes."""
all_dishes[10].introduction = """
Mapo Tofu is a flavorful Sichuan dish loved for its rich and silky tofu in a
spicy chili bean sauce. With a delightful numbing sensation from Sichuan
peppercorns, it's a true example of the province's signature bold and complex
flavors."""
all_dishes[11].introduction = """
Sweet and Sour Pork is a popular dish that combines crispy, tender pieces of
pork with a vibrant sweet and tangy sauce. Bell peppers, onions, and pineapple
add colorful bursts of flavor, making it a delightful choice for those seeking
a balance of tastes."""
all_dishes[12].introduction = """
General Tso's Chicken is a beloved American-Chinese dish featuring battered and
fried chicken pieces tossed in a sweet and slightly spicy sauce. It's a perfect
blend of textures and flavors, creating a deliciously addictive combination."""
all_dishes[13].introduction = """
Mongolian Beef is a stir-fry dish that showcases tender slices of beef coated
in a savory-sweet sauce. It's often accompanied by green onions and a touch of
garlic, offering a satisfying medley of tastes with a hint of smokiness."""
all_dishes[14].introduction = """
Ma Po Tofu is a dish that highlights soft tofu cubes in a spicy and aromatic
sauce. Ground meat and fermented bean paste lend depth to the flavors, while
Sichuan peppercorns provide their signature numbing effect, making it an
unforgettable culinary experience."""
all_dishes[15].introduction = """
Crispy Duck is a Chinese delicacy featuring tender duck meat encased in a crispy
skin. Often served with pancakes, scallions, and hoisin sauce, it's a delightful
dish with a satisfying contrast between textures."""
all_dishes[16].introduction = """
Szechuan Beef is a flavorful dish that showcases thinly sliced beef stir-fried
with a vibrant mix of vegetables and a spicy Sichuan sauce. Its fiery flavors
and aromatic spices make it a standout choice for those seeking a bold culinary
adventure."""
all_dishes[17].introduction = """
Broccoli Beef is a simple yet satisfying dish that combines marinated slices of
beef with crisp broccoli florets in a savory sauce. The combination of tender
beef and vibrant greens makes it a popular choice for a well-rounded meal."""
all_dishes[18].introduction = """
Chicken with Cashew Nuts is a delightful stir-fry that features tender chicken,
crunchy cashews, and colorful vegetables in a light sauce. The contrast between
the juicy chicken, nutty cashews, and fresh veggies creates a delightful harmony
of textures."""
all_dishes[19].introduction = """
Orange Chicken is a sweet and tangy dish that pairs crispy battered chicken with
a zesty orange-flavored sauce. The citrusy notes add a refreshing twist to the
savory flavors, making it a favorite among those who enjoy a burst of brightness
in their meal."""

In [ ]:
def show_all_dishes(all_dishes):
  for dish in all_dishes:
      print(f"Name: {dish.name}")
      print(f"Price: ${dish.price}")
      print(f"Spicy: {'Yes' if dish.is_spicy else 'No'}")
      print(f"Category: {dish.category}")
      print(f"nuts contained: {'Yes' if dish.is_nuts else 'No'}")
      print(f"lactose contained: {'Yes' if dish.is_lact else 'No'}")
      print(f"Is this dish suitable for vegetarians: {'Yes' if dish.is_vege else 'No'}")
      print(f"Introduction: {dish.introduction}")
      print("-" * 20)

In [ ]:
def show_all_dishes_partial_info(all_dishes):
  for dish in all_dishes:
      print(f"Name: {dish.name}")
      print(f"Price: ${dish.price}")
      print(f"Spicy: {'Yes' if dish.is_spicy else 'No'}")
      print(f"Category: {dish.category}")
      print(f"Introduction: {dish.introduction}")
      print("-" * 20)

In [ ]:
#customer class
class Customer:
    def __init__(self, age, gender, nationality, group_size, love_spicy):
        self.age = age
        self.gender = gender
        self.nationality = nationality
        self.group_size = group_size
        self.love_spicy = love_spicy
class CustomerFactory:
    def create_cus(self, age, gender, nationality, group_size, love_spicy):
        return Dish(age, gender, nationality, group_size, love_spicy)

In [ ]:
#function to transfer customer dictionary into dataframe
#customer_data = []

temp = {
    'Sam': [50, 'Male', 'India', 3, True]
}

def customer_trans(customer_info):
  customer_data = []
  for name,info in customer_info.items():
    age,gender,nationality,group_size,love_spicy = info
    age_group = ""
    if 10 <= int(age) <= 30:
      age_group = "1"
    elif 30 < int(age) <= 50:
      age_group = "2"
    else:
      age_group = "3"
  age_gender_nationality = f"{gender}|{nationality}|{age_group}|{'True' if love_spicy else 'False'}"
  customer_data.append([name, age_gender_nationality])

  columns = ["Customer Name", "Information"]
  df_customer = pd.DataFrame(customer_data, columns=columns)
  return df_customer

aaa = customer_trans(temp)
print(aaa)

  Customer Name        Information
0           Sam  Male|India|2|True


In [ ]:
#calculate the similarity

def cal_similarity(df_customer):
  #print(df_customer)
  #bugs here
  #same value added into dataframe every time
  combined_data = pd.concat([d_food_attri["attribute"], df_customer["Information"]])

  vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
  info_matrix = vectorizer.fit_transform(combined_data)

  cosine_similarities = cosine_similarity(info_matrix[-1], info_matrix[:-1])
  #print(cosine_similarities)

  d_food_attri['cosine_similarity'] = cosine_similarities[0][:20]
  food_cus_similarity = d_food_attri.sort_values('cosine_similarity',ascending=False)
  #print(food_cus_similarity)
  return food_cus_similarity.index.tolist()

temp_df = customer_trans(temp)
cal_similarity(temp_df)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['Beef Chow Fun',
 'Ma Po Tofu',
 'Wonton Soup',
 'Vegetable Fried Rice',
 'Szechuan Beef',
 'Shrimp Lo Mein',
 'Orange Chicken',
 'Mapo Tofu',
 'Egg Drop Soup',
 'Dan Dan Noodles',
 'Yangzhou Fried Rice',
 'Kung Pao Chicken',
 'Broccoli Beef',
 'Mongolian Beef',
 "General Tso's Chicken",
 'Sweet and Sour Pork',
 'Crispy Duck',
 'Chicken with Cashew Nuts',
 'Hot and Sour Soup',
 'Pineapple Fried Rice']

In [ ]:
#fisrt screening dishes, using diet_restrictions, vegetarian_preference
#temp_menu_list = cal_similarity(aaa)
def diet_screening(temp_menu_list,vegetarian_preference,group_size,budget,nuts_allergic,lactose_intolerance):
  dish_instance_map = {dish.name: dish for dish in all_dishes}

  ordered_dish_instances = [dish_instance_map[dish_name] for dish_name in temp_menu_list]

  #show_all_dishes(ordered_dish_instances)

  filtered_dish_instances = []
  for dish_instance in ordered_dish_instances:
    if (vegetarian_preference == True and dish_instance.is_vege == False) or \
      (nuts_allergic == True and dish_instance.is_nuts == True) or \
      (lactose_intolerance ==True and dish_instance.is_lact == True):
      continue  # Skip this dish instance
    else:
      filtered_dish_instances.append(dish_instance)
      if sum(dish_instance.price for dish_instance in filtered_dish_instances) >= float(budget) + float(group_size)*average_price/2:
        break
      if len(filtered_dish_instances) == int(group_size) + 2:
        break
  ordered_dish_names = [dish.name for dish in filtered_dish_instances]
  return filtered_dish_instances

#  for dish_instance in ordered_dish_instances:
#    print(f"Dish: {dish_instance.name}")
#diet_screening(temp_menu_list,False,5,70,False,False)

In [ ]:
from sqlalchemy.sql.expression import null
#2.chatbot scenario
import random
import pandas as pd

customer_info = {}
comments = []

#this is for customer data collection

#all_data_collect = pd.DataFrame(columns=['name', 'age', 'gender', 'nationality', 'group_size', 'love_spicy', 'favorite_dishes', 'total_billing', 'comment', 'date'])
all_data_dict = {}

def ask_age(name):
    while True:
        response = input(f"Nice to meet you, {name.title()}. May I know your age? (You can also say 'prefer not to disclose'): ").lower()
        if response == 'prefer not to disclose':
            return None
        elif response.isdigit():
            return response
        else:
            print("Please enter a valid age or 'prefer not to disclose'.")

def ask_gender():
    while True:
        response = input("What's your gender? (Male, Female, Others, prefer not to disclose'): ").lower()
        if response in ['male', 'female', 'others']:
            return response
        elif response == 'prefer not to disclose':
            return None
        else:
            print("Please enter a valid gender or 'prefer not to disclose'.")

def ask_nationality():
    while True:
        nationality = input("Thank you. What's your nationality? ")
        if nationality.strip() and not any(char.isdigit() for char in nationality):
            return nationality
        else:
            print("Please enter a valid non-numeric nationality.")

def ask_spicy_preference():
    while True:
        response = input("Do you enjoy spicy food? Please type 'yes' or 'no': ").lower()
        if response == 'yes' or response == 'no':
            return response
        else:
            print("Please enter 'yes' or 'no'.")

def ask_group_size():
    while True:
        try:
            group_size = int(input("How many people are in your group? "))
            return group_size
        except ValueError:
            print("Please enter a valid number.")

def ask_vegetarian_preference():
    while True:
        response = input("Are you a vegetarian? Please type 'yes' or 'no': ").lower()
        if response == 'yes' or response == 'no':
            return response
        else:
            print("Please enter 'yes' or 'no'.")

def ask_budget():
    while True:
        try:
            budget = int(input("Lastly, what's your budget in CAD for this meal? "))
            return budget
        except ValueError:
            print("Please enter a valid number.")

def ask_nut_allergic():
    while True:
        response = input("Are you allergic to nuts? Please type 'yes' or 'no': ").lower()
        if response == 'yes' or response == 'no':
            return response
        else:
            print("Please enter 'yes' or 'no'.")

def ask_lactose_intolerance():
    while True:
        response = input("Are you lactose intolerance? Please type 'yes' or 'no': ").lower()
        if response == 'yes' or response == 'no':
            return response
        else:
            print("Please enter 'yes' or 'no'.")

def recommend_dishes(name,recommend_insance_list):

    recommendation = f'''
Dear {name.title()}, based on your input, the recommended dishes for you are shown above
{show_all_dishes_partial_info(recommend_insance_list)}'''

    return recommendation

def Recommendation_scenario():
    name = input("Please provide your name: ")

    age = int(ask_age(name))
    gender = ask_gender()
    nationality = ask_nationality()
    love_spicy = ask_spicy_preference()
#-------------------------------------------------------------------------------------
    #diet_restrictions = input("Are there any dietary restrictions I should be aware of? Please list them, if any. ")
    nuts_allergic = ask_nut_allergic()

    lactose_intolerance = ask_lactose_intolerance()

    vegetarian_preference = ask_vegetarian_preference()
    group_size = ask_group_size()
    budget = ask_budget()

#-------------------------------------------------------------------------------------
    customer_info[name] = [age, gender, nationality, group_size, love_spicy]
    #customer_info = [age, gender, nationality, group_size, love_spicy]
    customer_info_df = customer_trans(customer_info)
    similarity_rank_list = cal_similarity(customer_info_df)
    recommend_insance_list = diet_screening(similarity_rank_list,vegetarian_preference, group_size, budget, nuts_allergic,lactose_intolerance)

    dishes = recommend_dishes(name,recommend_insance_list)
    #print(dishes)

def comment_scenario():
    while True:
        current_date = datetime.today().strftime('%Y-%m-%d')
        current_date = pd.to_datetime('today').strftime('%Y-%m-%d')
        temp = []
        all_data_collect = pd.DataFrame(columns=['name', 'age', 'gender', 'nationality', 'group_size', 'love_spicy', 'favorite_dishes', 'total_billing', 'comment', 'date'])
        response = input("Have you eaten at our restaurant before? (yes/no): ").lower()
        if response == 'no':
            name = input("Please provide your name: ")
            comment = input("Please leave your comment: ")
            comments.append((name, comment))
            print(f"""
Dear {name.title()}, thank you for your comment. Here is your comment record:
'{comment}'
Have a great day!
""")
            return
        elif response == 'yes':
            collect_info = input("Can we ask you a few questions before your comment? (yes/no): ").lower()
            if collect_info == 'yes':
                name = input("Please provide your name: ")
                age = ask_age(name)
                gender = ask_gender()
                nationality = ask_nationality()
                group_size = ask_group_size()
                love_spicy = ask_spicy_preference()
                favorite_dishes = input("What are your favorite Chinese dishes? ")
                total_billing = float(input("What was the total billing amount? "))
                comment = input("Please leave your comment: ")
                print(f"""
Dear {name.title()}, thank you for your comment. Here is your comment record:
'{comment}'
Have a great day!
""")
                customer_data_collect = { 'name': name,
                              'age': age,
                              'gender': gender,
                              'nationality': nationality,
                              'group_size': group_size,
                              'love_spicy': love_spicy,
                              'favorite_dishes': favorite_dishes,
                              'total_billing': total_billing,
                              'comment': comment,
                              'date':current_date}
                all_data_collect = all_data_collect.append(customer_data_collect, ignore_index=True)
                excel_file = home_dir + 'all_data_collect.xlsx'
                existing_data = pd.read_excel(excel_file)
                appended_data = existing_data.append(all_data_collect, ignore_index=True)
                book = load_workbook(excel_file)
                writer = pd.ExcelWriter(excel_file, engine='openpyxl')
                writer.book = book
                appended_data.to_excel(writer, index=False, sheet_name='Sheet1')
                writer.save()
                return
            elif collect_info == 'no':
                name = input("Please provide your name: ")
                comment = input("Please leave your comment: ")
                comments.append((name, comment))
                print(f"""
                Dear {name.title()}, thank you for your comment. Here is your comment record:
                '{comment}'
                Have a great day!
                """)
                customer_data_collect = { 'name': name,
                              'age': None,
                              'gender':None,
                              'nationality': None,
                              'group_size': None,
                              'love_spicy': None,
                              'favorite_dishes': None,
                              'total_billing': None,
                              'comment': comment,
                              'date':current_date}
                all_data_collect = all_data_collect.append(customer_data_collect, ignore_index=True)
                excel_file = home_dir + 'all_data_collect.xlsx'
                existing_data = pd.read_excel(excel_file)
                appended_data = existing_data.append(all_data_collect, ignore_index=True)
                book = load_workbook(excel_file)
                writer = pd.ExcelWriter(excel_file, engine='openpyxl')
                writer.book = book
                appended_data.to_excel(writer, index=False, sheet_name='Sheet1')
                writer.save()
                return
            else:
                print("Please enter 'yes' or 'no'.")
        else:
            print("Please enter 'yes' or 'no'.")

    customer_info[name] = [age, gender, nationality, group_size, love_spicy, favorite_dishes, total_billing]
    comments.append((name, comment))

def promotion_scenario():
    new_member_code = random.randint(00,99)
    while True:
        response = input("Are you interested in knowing about next week's promotion? (yes/no): ").lower()
        if response == 'yes':
            promotion_day = random.choice(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
            print(f"Next week's promotion will be on {promotion_day}. If you spend more than 50 CAD, you can have an 5% discount on the meal.")
            member = input("Our members will get additional discount. Are you a member of our restaurant? (yes/no)").lower()
            if member == 'yes':
              print(f"""
Welcome back, our beloved member.
Your additional discount is 5%.
In total, you will be having a 10% off on your purchase of 50 CAD and more on {promotion_day}.""")
              return
            elif member == 'no':
              new_member = input("Are you interested in becoming our members? (yes/no)").lower()
              if new_member == 'yes':
                print(f"""
Thank you for your interest.
Here is your new member discount code: NEWMEMBER{new_member_code}.
Please show this code when you are checking out.
Your member ID will be printed on your next receipt once you used the code.""")
                return
              elif new_member == 'no':
                print("Thank you for visiting. Have a good day!")
                return
              else:
                print("Please enter 'yes' or 'no'.")
            else:
              print("Please enter 'yes' or 'no'.")
        elif response == 'no':
            print("Thank you for visiting. Have a great day!")
            return
        else:
            print("Please enter 'yes' or 'no'.")

def order_scenario():
    dish_numbers = {i + 1: dish for i, dish in enumerate(dish_prices)}

    print("Welcome to our ordering system. Here's the menu:")
    menu_df = pd.DataFrame(list(dish_prices.items()), columns=["Dish", "Price"])
    menu_df.index += 1

    print(menu_df)
    order = {}
    while True:
        try:
            dish_num = int(input("Please enter the number of the dish you'd like to order (or 0 to finish): "))
            if dish_num == 0:
                break
            elif dish_num in dish_numbers:
                dish_name = dish_numbers[dish_num]
                quantity = int(input(f"How many {dish_name} would you like to order? "))
                if dish_name in order:
                    order[dish_name] += quantity
                else:
                    order[dish_name] = quantity
                print(f"{quantity} {dish_name} added to your order.")
            else:
                print("Invalid dish number. Please choose from the menu.")
        except ValueError:
            print("Please enter a valid number.")

    total_cost = sum(order[dish] * dish_prices[dish] for dish in order)
    response = "Here's your order:\n"
    for dish_name, quantity in order.items():
        response += f"{dish_name}: {quantity} x ${dish_prices[dish_name]:.2f} = ${quantity * dish_prices[dish_name]:.2f}\n"
    response += f"Total cost: ${total_cost:.2f}"
    return response

def chatbot_scenario():
  user_response = False

  while not user_response:
        print("""
Hello! Welcome to our Chinese restaurant.
Please tell us how can we help you today:
1. Dish Recommendation
2. Promotion Information
3. Leaving Comment
4. Ordering Food
5. Nothing Much
""")

        response = input("Enter the number of your choice: ")

        if response == '1':
            Recommendation_scenario()
            user_response = True
        elif response == '2':
            promotion_info = promotion_scenario()
            if promotion_info:
                print(promotion_info)
            user_response = True
        elif response == '3':
            comment_scenario()
            user_response = True
        elif response == '4':
            order = order_scenario()
            print(order)
            user_response = True
        elif response == '5':
            print("Thank you for visiting. Have a great day!")
            user_response = True
        else:
            print("I'm sorry, I didn't understand your response. Please choose a valid option.")

if __name__ == "__main__":
    chatbot_scenario()



Hello! Welcome to our Chinese restaurant.
Please tell us how can we help you today:
1. Dish Recommendation
2. Promotion Information
3. Leaving Comment
4. Ordering Food
5. Nothing Much

Enter the number of your choice: 5
Thank you for visiting. Have a great day!
